In [1]:
import os
os.getcwd()

'/home/jupyteach-msda/jupyteach-ai/videos'

In [5]:
import os
import moviepy.editor as mp
from whisper.utils import get_writer
import whisper

def transcribe_video_to_srt(input_folder_video, output_folder_srt):
    # Check if the output folder exists, or create it if not
    if not os.path.exists(output_folder_srt):
        os.makedirs(output_folder_srt)

    # List all video files in the input folder
    video_files = [f for f in os.listdir(input_folder_video) if f.endswith('.mp4')]

    # Loop through each video file
    for video_file in video_files:
        # Convert the video to MP3 audio
        clip = mp.VideoFileClip(os.path.join(input_folder_video, video_file))
        audio_file = clip.audio
        audio_file.write_audiofile(os.path.join(output_folder_srt, video_file.replace(".mp4", ".mp3")))

        # Load the Whisper ASR model
        model = whisper.load_model("tiny")

        # Transcribe the audio
        audio_filename = os.path.join(output_folder_srt, video_file.replace(".mp4", ".mp3"))
        result = model.transcribe(audio_filename)

        # Define SRT writer options
        srt_writer = get_writer("srt", output_folder_srt)
        options = {
            'max_line_width': None,
            'max_line_count': None,
            'highlight_words': False
        }

        # Write the transcription to an SRT file
        srt_writer(result, audio_filename, options)

# Specify input and output folders
input_folder_video = "Test_10182023"
output_folder_srt = "Test_srt_output_10182023"

# Call the function to transcribe and save the SRT files for all videos in the input folder
transcribe_video_to_srt(input_folder_video, output_folder_srt)

########################################################################################

import os
import re
rex = re.compile(r"\d+\n(\d{2}:\d{2}:\d{2}),\d{3} --> (\d{2}:\d{2}:\d{2}),\d{3}")

def chunk_srt_files(full_text, chunk_length):

    # split on the regex.
    splits = rex.split(full_text)[1:]

    # combine parts into a list of 3-tuples (start, end, txt)
    parts = []
    for i in range(0, len(splits), 3):
        start_time = splits[i]
        end_time = splits[i+1]
        content = splits[i+2].strip()
        parts.append((start_time, end_time, content))
        

    # combine multiple parts to get desired chunk length
    # will be a list of 3-tuples (start, end, txt)
    chunks = []
    ix = 0
    current_chunk_text = ""
    for i, part in enumerate(parts):
        current_chunk_text = current_chunk_text + " " + part[2]
        if len(current_chunk_text) > chunk_length or i == len(parts) - 1:
            # if we have a long enough chunk OR we are on the last piece of content...
            current_chunk = (
                parts[ix][0],  # starting timestamp
                part[1],
                current_chunk_text.strip()
            )
            chunks.append(current_chunk)
            ix = i  # we repeat this chunk one more time for overlap
            current_chunk_text =  part[2]

    return chunks

def process_all_srt_files(input_folder_srt, output_folder_chunk, chunk_length):
    # Ensure the output folder exists, or create it if not
    if not os.path.exists(output_folder_chunk):
        os.makedirs(output_folder_chunk)

    # List all SRT files in the input folder
    srt_files = [f for f in os.listdir(input_folder_srt) if f.endswith('.srt')]

    for srt_file in srt_files:
        # Read the content of the SRT file
        with open(os.path.join(input_folder_srt, srt_file)) as f:
            txt = f.read()

        # Call the chunk_srt_files function to process the SRT content
        chunks = chunk_srt_files(txt, chunk_length)

        # Save the chunked content to the output folder
        output_file = os.path.join(output_folder_chunk, srt_file)
        with open(output_file, 'w') as f:
            for chunk in chunks:
                f.write(f"{chunk[0]} --> {chunk[1]}\n{chunk[2]}\n\n")

# Set the paths for the input and output folders and specify the chunk length
input_folder_srt = output_folder_srt
output_folder_chunk = "Test_chunk_output_10182023"
chunk_length = 500

# Call the function to process all SRT files in the input folder
process_all_srt_files(input_folder_srt, output_folder_chunk, chunk_length)



MoviePy - Writing audio in Test_srt_output_10182023/2.1.2 pandas core functionality.mp3


MoviePy - Done.


NameError: name 'srt_writer' is not defined